# City Grid Scanning: Systematic Urban Coverage with UCID

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ucid-foundation/ucid/blob/main/notebooks/01_citygrid_scan_basics.ipynb)

---

## Overview

This notebook demonstrates how to systematically scan entire cities using H3 hexagonal grids and generate UCIDs for comprehensive urban coverage. You will learn:

1. Understanding H3 hexagonal spatial indexing
2. Generating city-wide grids from bounding boxes
3. Batch UCID generation for grid cells
4. Exporting grid data for analysis

### Prerequisites

- Completion of `00_ucid_basics.ipynb`
- Understanding of coordinate systems

---

In [ ]:
# Install dependencies
%pip install -q ucid h3 geopandas shapely

In [ ]:
# Imports
import geopandas as gpd
import h3
import numpy as np
import pandas as pd
from shapely.geometry import Polygon

import ucid
from ucid import create_ucid
from ucid.spatial import generate_grid_h3, scan_city_grid

print(f"UCID version: {ucid.__version__}")
print(f"H3 version: {h3.__version__}")

---

## 1. H3 Hexagonal Grid Basics

### 1.1 Understanding H3 Resolutions

H3 provides 16 resolution levels (0-15), each with different cell sizes:

| Resolution | Avg. Area (km²) | Avg. Edge (m) | Use Case |
|------------|-----------------|---------------|----------|
| 7 | 5.16 | 1,406 | Regional |
| 8 | 0.74 | 531 | City-wide |
| 9 | 0.11 | 201 | Neighborhood |
| 10 | 0.015 | 76 | Block |
| 11 | 0.002 | 29 | Building |

In [ ]:
# Get H3 cell for a specific location
lat, lon = 41.0082, 28.9784  # Istanbul

print("H3 Cells at Different Resolutions:")
print("=" * 50)
for res in range(7, 12):
    cell = h3.latlng_to_cell(lat, lon, res)
    area = h3.cell_area(cell, unit="km^2")
    print(f"Resolution {res:2d}: {cell} (Area: {area:.4f} km²)")

### 1.2 Cell Geometry

In [ ]:
# Get cell boundary as polygon
cell = h3.latlng_to_cell(lat, lon, 9)
boundary = h3.cell_to_boundary(cell)

print(f"Cell: {cell}")
print(f"Center: {h3.cell_to_latlng(cell)}")
print(f"Boundary vertices: {len(boundary)}")

# Create Shapely polygon
polygon = Polygon([(lon, lat) for lat, lon in boundary])
print(f"Polygon area: {polygon.area:.8f} deg²")

---

## 2. Generating City Grids

### 2.1 From Bounding Box

In [ ]:
# Define Istanbul bounding box
istanbul_bbox = {
    "min_lat": 40.80,
    "max_lat": 41.30,
    "min_lon": 28.50,
    "max_lon": 29.50,
}

# Generate H3 grid
resolution = 8  # City-wide analysis
grid_cells = generate_grid_h3(
    bbox=(
        istanbul_bbox["min_lon"],
        istanbul_bbox["min_lat"],
        istanbul_bbox["max_lon"],
        istanbul_bbox["max_lat"],
    ),
    resolution=resolution,
)

print(f"Generated {len(grid_cells):,} H3 cells at resolution {resolution}")

### 2.2 Cell Statistics

In [ ]:
# Calculate grid statistics
areas = [h3.cell_area(cell, unit="km^2") for cell in list(grid_cells)[:100]]

print("Grid Statistics:")
print(f"  Total cells: {len(grid_cells):,}")
print(f"  Avg cell area: {np.mean(areas):.4f} km²")
print(f"  Est. total area: {len(grid_cells) * np.mean(areas):.2f} km²")

---

## 3. Batch UCID Generation

### 3.1 Generate UCIDs for Grid

In [ ]:
# Generate UCIDs for all grid cells
ucid_data = []

for cell in list(grid_cells)[:500]:  # Limit for demo
    lat, lon = h3.cell_to_latlng(cell)

    ucid_str = create_ucid(
        city="IST",
        lat=lat,
        lon=lon,
        timestamp="2026W02T14",
        context="15MIN",
    )

    ucid_data.append(
        {
            "h3_cell": cell,
            "latitude": lat,
            "longitude": lon,
            "ucid": ucid_str,
        }
    )

df = pd.DataFrame(ucid_data)
print(f"Generated {len(df)} UCIDs")
df.head()

### 3.2 Using scan_city_grid()

In [ ]:
# Use the built-in city scanning function
city_grid = scan_city_grid(
    city="IST",
    resolution=8,
    context="15MIN",
    timestamp="2026W02T14",
    limit=1000,  # Limit for demo
)

print(f"Scanned {len(city_grid)} cells")
city_grid.head()

---

## 4. Creating GeoDataFrame

### 4.1 Add Geometries

In [ ]:
# Create geometries for each cell
def h3_to_polygon(h3_cell):
    """Convert H3 cell to Shapely polygon."""
    boundary = h3.cell_to_boundary(h3_cell)
    return Polygon([(lon, lat) for lat, lon in boundary])


df["geometry"] = df["h3_cell"].apply(h3_to_polygon)

# Convert to GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
print(f"GeoDataFrame created with {len(gdf)} features")
gdf.head()

### 4.2 Quick Visualization

In [ ]:
# Plot the grid (if matplotlib is available)
try:
    import matplotlib.pyplot as plt

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    gdf.plot(ax=ax, edgecolor="black", facecolor="lightblue", alpha=0.5)
    ax.set_title(f"Istanbul H3 Grid (Resolution {resolution})")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    plt.tight_layout()
    plt.show()
except ImportError:
    print("matplotlib not available for visualization")

---

## 5. Export Options

### 5.1 Export to GeoParquet

In [ ]:
# Export to GeoParquet (efficient columnar format)
output_path = "istanbul_grid.parquet"
gdf.to_parquet(output_path, index=False)
print(f"Exported to {output_path}")

### 5.2 Export to GeoJSON

In [ ]:
# Export to GeoJSON (for web applications)
geojson_path = "istanbul_grid.geojson"
gdf.to_file(geojson_path, driver="GeoJSON")
print(f"Exported to {geojson_path}")

---

## 6. Multi-Resolution Analysis

In [ ]:
# Compare cell counts at different resolutions
resolutions = [7, 8, 9, 10]
cell_counts = []

for res in resolutions:
    cells = generate_grid_h3(
        bbox=(28.5, 40.8, 29.5, 41.3),
        resolution=res,
    )
    cell_counts.append(len(cells))

comparison_df = pd.DataFrame(
    {
        "Resolution": resolutions,
        "Cell Count": cell_counts,
    }
)

print("Resolution Comparison:")
comparison_df

---

## Summary

In this notebook, you learned:

1. **H3 Basics**: Understanding hexagonal spatial indexing
2. **Grid Generation**: Creating city-wide H3 grids
3. **Batch Processing**: Generating UCIDs for all grid cells
4. **GeoDataFrames**: Working with spatial data in pandas
5. **Export**: Saving grids to GeoParquet and GeoJSON

### Next Steps

- **02_15min_city_isochrones.ipynb**: Analyze 15-minute city accessibility

---

*Copyright 2026 UCID Foundation. Licensed under EUPL-1.2.*